In [3]:
!pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MelanomaDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = pd.read_csv(os.path.join(root_dir, 'csv_files', csv_file))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_id = row['image_id']
        patch_id = row['Patch_id']  # Change to 'Patch_id' to match CSV
        label = row['label']

        try:
            patch_num = int(patch_id.split('_Patch_')[1].split('.')[0])
        except IndexError:
            print(f"Error processing patch_id: {patch_id}")
            raise

        patch_type = 'mel_patches' if label == 1 else 'bkl_patches'

        img_path = os.path.join(self.root_dir, 'Images', 'img_patches', patch_type, f'Patch_{patch_num}', patch_id)
        
        # Open the image
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        else:
            # If no transform is provided, convert to tensor and normalize
            image = transforms.ToTensor()(image)
            image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)

        return image, patch_num - 1, label  # patch_num - 1 for 0-based indexing

# Define transformations (only normalization, no resizing)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset and dataloader
dataset = MelanomaDataset('/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2', 'patches_with_labels.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.multiprocessing as mp
from torchvision import transforms
from mel_dataloader import MelanomaDataset
from tqdm import tqdm
import matplotlib.pyplot as plt

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels=3, patch_size=8, embed_dim=768):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)  # (B, embed_dim, 8, 8)
        return x.flatten(2).transpose(1, 2)  # (B, 64, embed_dim)

class PositionEmbedding(nn.Module):
    def __init__(self, num_patches=16, embed_dim=768):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

    def forward(self, x):
        return x + self.pos_embed

class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim=768, num_heads=12, num_layers=12, mlp_ratio=4, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, 
                                       dim_feedforward=embed_dim*mlp_ratio, 
                                       dropout=dropout, activation='gelu')
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

class MelanomaTransformer(nn.Module):
    def __init__(self, in_channels=3, patch_size=8, embed_dim=768, num_heads=12, num_layers=12, num_classes=1):
        super().__init__()
        self.patch_size = patch_size
        self.patch_embed = PatchEmbedding(in_channels, patch_size, embed_dim)
        self.pos_embed = nn.Parameter(torch.zeros(1, 1024, embed_dim))  # Max 256x256 image size
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        self.transformer = TransformerEncoder(embed_dim, num_heads, num_layers)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B, C, H, W = x.shape
        num_patches = (H // self.patch_size) * (W // self.patch_size)
        x = self.patch_embed(x)
        x = x + self.pos_embed[:, :num_patches]
        x = self.transformer(x)
        x = x.mean(dim=1)  # Global average pooling
        return self.fc(x)

def train_model(model, dataloader, num_epochs=50, lr=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    train_losses = []
    train_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Wrap the dataloader with tqdm
        for batch, positions, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch, labels = batch.to(device), labels.to(device).float()
            
            optimizer.zero_grad()
            outputs = model(batch).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            predicted = (outputs > 0).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(dataloader)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
    
    # Plot training loss and accuracy
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy Over Epochs')
    plt.legend()

    plt.show()

    return model

def main():
    # Set up the dataset and dataloader
    root_dir = '/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2'
    csv_file = 'patches_with_labels.csv'
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = MelanomaDataset(root_dir=root_dir, csv_file=csv_file, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, num_workers=4, shuffle=True)

    # Initialize and train the model
    model = MelanomaTransformer()
    trained_model = train_model(model, dataloader)

    # Save the trained model
    torch.save(trained_model.state_dict(), 'trained_melanoma_transformer.pth')

if __name__ == '__main__':
    mp.set_start_method('spawn')  # Set the start method for multiprocessing
    main()
